In [1]:
import pandas as pd
import numpy as np
import sys

In [2]:
df_train = pd.read_csv(r"C:\Users\maxim\Desktop\js\linfit\test\archive\mnist_train.csv")
df_test = pd.read_csv(r"C:\Users\maxim\Desktop\js\linfit\test\archive\mnist_test.csv")

In [3]:
xcolnams = df_train.columns.tolist()
print("check0")
df_train[xcolnams[1:]] /= 255. #convert the pixel values to 0..1
print("check1")

df_train["ones"] = 1.
xcolnams += ["ones"]
Xs_train = df_train[xcolnams[1:]].values.T
print("check2")

check0
check1
check2


In [4]:
ycolnams = []
for i in range(10):
    coln = "is"+str(i)
    ycolnams.append(coln)
    df_train[coln] = (df_train["label"] == i).astype(float)

In [5]:
xcolnams = df_test.columns.tolist()
print("check0")
df_test[xcolnams[1:]] /= 255. #convert the pixel values to 0..1
print("check1")

df_test["ones"] = 1.
xcolnams += ["ones"]
Xs_test = df_test[xcolnams[1:]].values.T
print("check2")

check0
check1
check2


In [6]:
ycolnams = []
for i in range(10):
    coln = "is"+str(i)
    ycolnams.append(coln)
    df_test[coln] = (df_test["label"] == i).astype(float)

In [7]:
Ys_train = df_train[ycolnams + ["ones"]].values.T

In [8]:
Ys_test = df_test[ycolnams + ["ones"]].values.T

In [9]:
sys.path.append(r"C:\Users\maxim\Desktop\js\linfit")
from linfit import lay, lay_holder

In [10]:
Ys_train.shape

(11, 60000)

In [11]:
model = lay_holder([lay(784,128), lay(128,10)]) 
#model = lay_holder([lay(784,10, False)])

In [12]:
NTR = 1
for j in range(NTR):
    print("Back")
    for i in range(len(model.layers)):
        model.solve_update(Xs_train, Ys_train, n=len(model.layers)-i)
        forw = model.forward(Xs_train)
        mse = np.mean((forw[0,:] - Ys_train[0,:])**2)
        print(mse)

    print("Front")
    for i in range(len(model.layers)):
        model.solve_update(Xs_train, Ys_train, n=i + 1)
        forw = model.forward(Xs_train)
        #print(forw.shape)
        mse = np.mean((forw[0,:] - Ys_train[0,:])**2)
        print(mse)

Back
doing back
    0.00 ns in backward_mult(<linfit.lay_holder obj..., 0)
doing mult
doing forward
  179.00 ms in forward(<linfit.lay object at ..., [[0. 0. 0. ... 0. 0. 0...)
  181.00 ms in forward(<linfit.lay_holder obj..., [[0. 0. 0. ... 0. 0. 0..., 1)
doing solve
(129, 60000)
(11, 60000)
  993.00 ms in appx_solve([[57.96558634 62.66793..., [[0. 1. 0. ... 0. 0. 0...)
    1.18 s in solve_lay(<linfit.lay_holder obj..., [[0. 0. 0. ... 0. 0. 0..., [[0. 1. 0. ... 0. 0. 0..., 2)
    0.00 ns in update(<linfit.lay object at ..., [[-4.30628933e-03 9.08...)
    1.18 s in solve_update(<linfit.lay_holder obj..., [[0. 0. 0. ... 0. 0. 0..., [[0. 1. 0. ... 0. 0. 0..., n=2)
  157.00 ms in forward(<linfit.lay object at ..., [[0. 0. 0. ... 0. 0. 0...)
   11.00 ms in forward(<linfit.lay object at ..., [[57.96558634 62.66793...)
  171.00 ms in forward(<linfit.lay_holder obj..., [[0. 0. 0. ... 0. 0. 0...)
0.024722895922232653
doing back
    0.00 ns in backward_mult(<linfit.lay_holder obj..., 1)
doing m

In [13]:
forw = model.forward(Xs_train)
# #print(forw.shape)
# mse = np.mean((forw[0,:] - Ys_train[0,:])**2)

guessed = np.argmax(forw[:-1,:], axis=0)
# print(mse)

  212.00 ms in forward(<linfit.lay object at ..., [[0. 0. 0. ... 0. 0. 0...)
   12.00 ms in forward(<linfit.lay object at ..., [[0.54530592 0. 0. ......)
  228.00 ms in forward(<linfit.lay_holder obj..., [[0. 0. 0. ... 0. 0. 0...)


In [14]:
sum(guessed == df_train["label"].values) / len(guessed)

0.8968333333333334

In [15]:
1/10

0.1

In [16]:
from sklearn.metrics import confusion_matrix

In [17]:
confusion_matrix(df_train["label"].values, guessed)

array([[5710,    4,   38,   10,    8,   57,   35,    7,   51,    3],
       [   2, 6486,   59,   22,   12,   22,    3,   19,  111,    6],
       [  64,   80, 5283,  110,   87,   13,  118,   56,  125,   22],
       [  27,   46,  198, 5269,   13,  239,   12,   91,  153,   83],
       [  11,   43,   43,    1, 5337,   29,   52,   13,   59,  254],
       [  89,   31,   52,  220,   36, 4527,  112,   40,  263,   51],
       [  65,   27,   69,    1,   32,   98, 5569,    3,   51,    3],
       [  26,   85,   86,   48,  102,   16,    2, 5707,   19,  174],
       [  40,  267,   88,  199,   62,  219,   49,   13, 4812,  102],
       [  51,   13,   28,   88,  305,   49,    4,  244,   57, 5110]],
      dtype=int64)

In [18]:
forw = model.forward(Xs_test)
# #print(forw.shape)
# mse = np.mean((forw[0,:] - Ys_train[0,:])**2)

guessed = np.argmax(forw[:-1,:], axis=0)
# print(mse)

   37.00 ms in forward(<linfit.lay object at ..., [[0. 0. 0. ... 0. 0. 0...)
    4.00 ms in forward(<linfit.lay object at ..., [[0. 1.50432816 0.4779...)
   44.00 ms in forward(<linfit.lay_holder obj..., [[0. 0. 0. ... 0. 0. 0...)


In [19]:
sum(guessed == df_test["label"].values) / len(guessed)

0.8946

In [20]:
confusion_matrix(df_test["label"].values, guessed)

array([[ 954,    0,    1,    1,    0,   10,    6,    3,    5,    0],
       [   0, 1098,    6,    5,    1,    1,    3,    1,   20,    0],
       [  11,   15,  906,   20,   11,    4,   21,   14,   26,    4],
       [   6,    2,   25,  887,    3,   34,    2,   17,   25,    9],
       [   1,   11,    4,    0,  902,    0,   14,    4,    8,   38],
       [  16,    5,    8,   43,    9,  735,   11,   15,   45,    5],
       [  15,    4,   10,    0,   11,   21,  887,    1,    9,    0],
       [   3,   20,   25,    9,   16,    0,    1,  904,    1,   49],
       [   9,   23,    9,   31,   17,   42,   13,    8,  812,   10],
       [  10,    4,    2,   14,   57,    9,    1,   37,   14,  861]],
      dtype=int64)